In [1]:
import sys
import os

root_dir = r"D:/BIN/Project/agentic_chatbot"
os.chdir(root_dir)
sys.path.append(root_dir)

In [ ]:
from app.graph.state import ChatState
from app.intent.base import Intent
from app.retrievers.registry import get_product_retriever
from langchain_core.runnables import RunnableLambda
from app.graph.chains.retrieve_products_chain import build_compare_table, retrieve_products_docs, retrieve_products_chain
from app.embeddings.embedding_manager import EmbeddingManager
from app.retrievers.vector_store import VectorStore
from app.retrievers.retriever import RAGRetriever
from app.retrievers.registry import init_retrievers
from app.graph.nodes.retrieve_info import retrieve_info_node
from app.graph.chains.retrieve_info_chain import retrieve_1_car, detect_car_fields, build_info_answer, render_car_fields

c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
embedding_manager = EmbeddingManager()
policy_store = VectorStore("policy")
product_store = VectorStore("product")

if policy_store.collection.count() == 0 or product_store.collection.count() == 0:
    raise RuntimeError("Vector store is empty. Run load_all_collections() to load")

policy_retriever = RAGRetriever(policy_store, embedding_manager)
product_retriever = RAGRetriever(product_store, embedding_manager)

init_retrievers(policy_retriever=policy_retriever, product_retriever=product_retriever)

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384
Vector store initialized. Collection: policy
Existing documents in collection: 8
Vector store initialized. Collection: product
Existing documents in collection: 30


In [ ]:
compare_state = ChatState("tôi đang đang phân vân giữa Volkswagen Tiguan và Toyota Land Cruiser Prado", Intent.COMPARE_CARS, [], "", [], [], "", "")

In [ ]:
list_2_car = retrieve_products_docs(compare_state).compared_car

Retrieving documents for query: tôi đang đang phân vân giữa Volkswagen Tiguan và Toyota Land Cruiser Prado
Top k: 2, Score threshold: 0.3
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.81it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


In [ ]:
list_2_car

[{'id': 'doc_63427c5a_26',
  'content': 'Toyota Land Cruiser Prado là SUV cao cấp sản xuất năm 2022, nhập khẩu từ Nhật Bản, sử dụng động cơ xăng 2.7L, hộp số tự động và cấu hình 7 chỗ ngồi. Xe có giá 4,5 tỷ đồng, phù hợp cho người dùng cần SUV cỡ lớn sử dụng đa địa hình.',
  'metadata': {'body_type': 'SUV',
   'engine': '2.7L',
   'origin': 'Nhật Bản',
   'price_vnd': 4500000000,
   'fuel': 'Xăng',
   'brand': 'Toyota',
   'year': 2022,
   'seats': 7,
   'content_length': 230,
   'doc_index': 26,
   'transmission': 'Tự động',
   'model': 'Land Cruiser Prado',
   'segment': 'SUV cao cấp',
   'id': 27},
  'similarity_score': 0.5379429161548615,
  'distance': 0.46205708384513855,
  'rank': 1},
 {'id': 'doc_e09f7380_27',
  'content': 'Volkswagen Tiguan là SUV phân khúc C đời 2023, nhập khẩu từ Đức, trang bị động cơ xăng 2.0L tăng áp, hộp số tự động và cấu hình 7 chỗ ngồi. Xe có giá 1,999 tỷ đồng, phù hợp cho gia đình cần SUV châu Âu cỡ trung.',
  'metadata': {'year': 2023,
   'engine': '2.

In [ ]:
list_2_car[0]['metadata']['brand']

'Toyota'

In [ ]:
print(type(list_2_car))
print(type(list_2_car[0]))

<class 'list'>
<class 'dict'>


In [ ]:
compare_state = retrieve_products_docs(compare_state)
compare_state = build_compare_table(compare_state)

Retrieving documents for query: tôi đang đang phân vân giữa Volkswagen Tiguan và Toyota Land Cruiser Prado
Top k: 2, Score threshold: 0.3
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 42.25it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


In [ ]:
print(compare_state.response)

Đây là bảng so sánh thông số giữa Toyota Land Cruiser Prado và Volkswagen Tiguan:
+----------------+---------------------------+-------------------+
| Thông số       | Toyota Land Cruiser Prado | Volkswagen Tiguan |
+----------------+---------------------------+-------------------+
| body_type      | SUV                       | SUV               |
| brand          | Toyota                    | Volkswagen        |
| content_length | 230                       | 211               |
| doc_index      | 26                        | 27                |
| engine         | 2.7L                      | 2.0L Turbo        |
| fuel           | Xăng                      | Xăng              |
| id             | 27                        | 28                |
| model          | Land Cruiser Prado        | Tiguan            |
| origin         | Nhật Bản                  | Đức               |
| price_vnd      | 4500000000                | 1999000000        |
| seats          | 7                         | 

In [ ]:
info_state = ChatState("tôi đang thắc mắc về số chỗ ngồi và nhiên liệu của xe Toyota Land Cruiser Prado", Intent.ASK_CAR_INFO, [], "", [], [], "", "")
car = retrieve_1_car(info_state.user_message)[0]
print(car)
print(type(car))

Retrieving documents for query: tôi đang thắc mắc về số chỗ ngồi và nhiên liệu của xe Toyota Land Cruiser Prado
Top k: 1, Score threshold: 0.3
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 42.41it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)
{'id': 'doc_63427c5a_26', 'content': 'Toyota Land Cruiser Prado là SUV cao cấp sản xuất năm 2022, nhập khẩu từ Nhật Bản, sử dụng động cơ xăng 2.7L, hộp số tự động và cấu hình 7 chỗ ngồi. Xe có giá 4,5 tỷ đồng, phù hợp cho người dùng cần SUV cỡ lớn sử dụng đa địa hình.', 'metadata': {'brand': 'Toyota', 'content_length': 230, 'origin': 'Nhật Bản', 'transmission': 'Tự động', 'year': 2022, 'doc_index': 26, 'id': 27, 'model': 'Land Cruiser Prado', 'fuel': 'Xăng', 'segment': 'SUV cao cấp', 'engine': '2.7L', 'seats': 7, 'price_vnd': 4500000000, 'body_type': 'SUV'}, 'similarity_score': 0.5586130023002625, 'distance': 0.44138699769973755, 'rank': 1}
<class 'dict'>


In [ ]:
fields = detect_car_fields(info_state.user_message)
print(fields)

['fuel', 'seats']


In [ ]:
render = render_car_fields(fields, car['metadata'])
print(render)

- fuel: Xăng
- seats: 7 chỗ


In [ ]:
test = ChatState("tôi đang thắc mắc về số chỗ ngồi và nhiên liệu của xe Toyota Land Cruiser Prado", Intent.ASK_CAR_INFO, [], "", [], [], "", "")
print(build_info_answer(test).response)

Retrieving documents for query: tôi đang thắc mắc về số chỗ ngồi và nhiên liệu của xe Toyota Land Cruiser Prado
Top k: 1, Score threshold: 0.3
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 35.72it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)
Thông tin mà bạn đang thắc mắc về xe Toyota Land Cruiser Prado đời 2022 như sau:
- fuel: Xăng
- seats: 7 chỗ


In [ ]:
from app.embeddings.embedding_manager import PROD_DOCS

print(PROD_DOCS)
print(len(PROD_DOCS))
print(type(PROD_DOCS))
print(PROD_DOCS[0])
print(type(PROD_DOCS[0]))

[Document(metadata={'id': 1, 'brand': 'Toyota', 'model': 'Vios', 'segment': 'Hạng B', 'year': 2022, 'price_vnd': 458000000, 'body_type': 'Sedan', 'engine': '1.5L', 'fuel': 'Xăng', 'transmission': 'Tự động', 'seats': 5, 'origin': 'Thái Lan'}, page_content='Toyota Vios là mẫu sedan hạng B sản xuất năm 2022, sử dụng động cơ xăng 1.5L kết hợp hộp số tự động, cấu hình 5 chỗ ngồi. Xe được nhập khẩu từ Thái Lan, có mức giá 458 triệu đồng, phù hợp cho nhu cầu đi lại hàng ngày, sử dụng trong đô thị và gia đình nhỏ.'), Document(metadata={'id': 2, 'brand': 'Toyota', 'model': 'Corolla Cross', 'segment': 'SUV-C', 'year': 2023, 'price_vnd': 820000000, 'body_type': 'SUV', 'engine': '1.8L Hybrid', 'fuel': 'Hybrid', 'transmission': 'Tự động', 'seats': 5, 'origin': 'Thái Lan'}, page_content='Toyota Corolla Cross là mẫu SUV phân khúc C đời 2023, sử dụng hệ truyền động hybrid gồm động cơ 1.8L kết hợp mô-tơ điện, hộp số tự động và cấu hình 5 chỗ ngồi. Xe nhập khẩu Thái Lan, giá bán 820 triệu đồng, hướng tớ

In [ ]:
test_car = PROD_DOCS[0].__dict__
print(test_car)
print(type(test_car))

{'id': None, 'metadata': {'id': 1, 'brand': 'Toyota', 'model': 'Vios', 'segment': 'Hạng B', 'year': 2022, 'price_vnd': 458000000, 'body_type': 'Sedan', 'engine': '1.5L', 'fuel': 'Xăng', 'transmission': 'Tự động', 'seats': 5, 'origin': 'Thái Lan'}, 'page_content': 'Toyota Vios là mẫu sedan hạng B sản xuất năm 2022, sử dụng động cơ xăng 1.5L kết hợp hộp số tự động, cấu hình 5 chỗ ngồi. Xe được nhập khẩu từ Thái Lan, có mức giá 458 triệu đồng, phù hợp cho nhu cầu đi lại hàng ngày, sử dụng trong đô thị và gia đình nhỏ.', 'type': 'Document'}
<class 'dict'>


In [ ]:
print(test_car['metadata'])
print(test_car['page_content'])
print(test_car['metadata']['price_vnd'])

{'id': 1, 'brand': 'Toyota', 'model': 'Vios', 'segment': 'Hạng B', 'year': 2022, 'price_vnd': 458000000, 'body_type': 'Sedan', 'engine': '1.5L', 'fuel': 'Xăng', 'transmission': 'Tự động', 'seats': 5, 'origin': 'Thái Lan'}
Toyota Vios là mẫu sedan hạng B sản xuất năm 2022, sử dụng động cơ xăng 1.5L kết hợp hộp số tự động, cấu hình 5 chỗ ngồi. Xe được nhập khẩu từ Thái Lan, có mức giá 458 triệu đồng, phù hợp cho nhu cầu đi lại hàng ngày, sử dụng trong đô thị và gia đình nhỏ.
458000000


In [ ]:
import re
from typing import Dict, Optional

def parse_price_range(message: str) -> Dict[str, Optional[int]]:
    msg = message.lower()

    price_min = None
    price_max = None

    def to_vnd(value: float, unit: str) -> int:
        if unit in ["tỷ", "ty"]:
            return int(value * 1_000_000_000)
        return int(value * 1_000_000)
    
    match = re.search(
        r"(tầm|khoảng|loan quanh|trên dưới)\s+(\d+(?:[.,]\d+)?)\s*(triệu|tr|tỷ|ty)",
        msg
    )
    if match:
        center = to_vnd(float(match.group(2).replace(",", ".")), match.group(3))
        delta = int(center * 0.1)
        return {
            "price_min": center - delta,
            "price_max": center + delta
        }

    match = re.search(
        r"từ\s+(\d+(?:[.,]\d+)?)\s*(triệu|tr|tỷ|ty)\s+đến\s+(\d+(?:[.,]\d+)?)\s*(triệu|tr|tỷ|ty)",
        msg
    )
    if match:
        price_min = to_vnd(float(match.group(1).replace(",", ".")), match.group(2))
        price_max = to_vnd(float(match.group(3).replace(",", ".")), match.group(4))
        return {"price_min": price_min, "price_max": price_max}

    match = re.search(
        r"(dưới|không quá|tối đa)\s+(\d+(?:[.,]\d+)?)\s*(triệu|tr|tỷ|ty)",
        msg
    )
    if match:
        price_max = to_vnd(float(match.group(2).replace(",", ".")), match.group(3))
        return {"price_min": None, "price_max": price_max}

    match = re.search(
        r"(trên|từ)\s+(\d+(?:[.,]\d+)?)\s*(triệu|tr|tỷ|ty)",
        msg
    )
    if match:
        price_min = to_vnd(float(match.group(2).replace(",", ".")), match.group(3))
        return {"price_min": price_min, "price_max": None}

    match = re.search(
        r"(\d+(?:[.,]\d+)?)\s*(triệu|tr|tỷ|ty)",
        msg
    )
    if match:
        center = to_vnd(float(match.group(1).replace(",", ".")), match.group(2))
        delta = int(center * 0.1)
        return {
            "price_min": center - delta,
            "price_max": center + delta
        }

    return {"price_min": None, "price_max": None}


In [ ]:
min_price, max_price = (parse_price_range("xe dưới 1 tỷ 300 triệu")).values()

In [ ]:
print(min_price)
print(type(min_price))
if not min_price:
    print(1111)

None
<class 'NoneType'>
1111


In [ ]:
from app.embeddings.embedding_manager import PROD_DOCS

cars_list = [item.__dict__ for item in PROD_DOCS]
print(len(cars_list))
print((cars_list[0]['metadata']))
print(cars_list)

30
{'id': 1, 'brand': 'Toyota', 'model': 'Vios', 'segment': 'Hạng B', 'year': 2022, 'price_vnd': 458000000, 'body_type': 'Sedan', 'engine': '1.5L', 'fuel': 'Xăng', 'transmission': 'Tự động', 'seats': 5, 'origin': 'Thái Lan'}
[{'id': None, 'metadata': {'id': 1, 'brand': 'Toyota', 'model': 'Vios', 'segment': 'Hạng B', 'year': 2022, 'price_vnd': 458000000, 'body_type': 'Sedan', 'engine': '1.5L', 'fuel': 'Xăng', 'transmission': 'Tự động', 'seats': 5, 'origin': 'Thái Lan'}, 'page_content': 'Toyota Vios là mẫu sedan hạng B sản xuất năm 2022, sử dụng động cơ xăng 1.5L kết hợp hộp số tự động, cấu hình 5 chỗ ngồi. Xe được nhập khẩu từ Thái Lan, có mức giá 458 triệu đồng, phù hợp cho nhu cầu đi lại hàng ngày, sử dụng trong đô thị và gia đình nhỏ.', 'type': 'Document'}, {'id': None, 'metadata': {'id': 2, 'brand': 'Toyota', 'model': 'Corolla Cross', 'segment': 'SUV-C', 'year': 2023, 'price_vnd': 820000000, 'body_type': 'SUV', 'engine': '1.8L Hybrid', 'fuel': 'Hybrid', 'transmission': 'Tự động', 's

In [2]:
from app.graph.chains.recommend_car_chain import consider_demand_car, parse_price_range, build_recommendation_info, render_table_from_list_dict

fields = ['fuel', 'transmission', 'segment']
ideal_car = {'fuel': 'điện', 'transmission': 'số tự động', 'segment': 'suv-e'}
user_message = "Tôi đang cần xe điện, số tự động, phân khúc hạng suv e, tầm giá khoảng 1.7 tỷ"
filted_car = consider_demand_car(fields, ideal_car, user_message)
print(filted_car)
print(len(filted_car))
print(type(filted_car))
print(type(filted_car[0]))

c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[{'id': 29, 'brand': 'VinFast', 'model': 'VF9', 'segment': 'SUV-E', 'year': 2023, 'price_vnd': 1700000000, 'body_type': 'SUV', 'engine': 'Điện', 'fuel': 'Điện', 'transmission': 'Tự động', 'seats': 7, 'origin': 'Việt Nam'}]
1
<class 'list'>
<class 'dict'>


In [3]:
print(render_table_from_list_dict(filted_car))

+----+---------+-------+---------+------+------------+-----------+--------+------+--------------+-------+----------+
| id | brand   | model | segment | year | price_vnd  | body_type | engine | fuel | transmission | seats | origin   |
+----+---------+-------+---------+------+------------+-----------+--------+------+--------------+-------+----------+
| 29 | VinFast | VF9   | SUV-E   | 2023 | 1700000000 | SUV       | Điện   | Điện | Tự động      | 7     | Việt Nam |
+----+---------+-------+---------+------+------------+-----------+--------+------+--------------+-------+----------+
